In [5]:
import icalendar
from enum import Enum, auto
from zoneinfo import ZoneInfo
from datetime import (
    datetime, date,
    timedelta, timezone
)

utc_8 = ZoneInfo('Etc/GMT+8')

In [6]:
ics_file_path = r'./data/test/basic.ics'

with open(ics_file_path, 'r', encoding='utf-8') as f:
    calendar = icalendar.Calendar.from_ical(f.read())

In [7]:
class StEventType:
    REGULAR = 'Regular'
    ALL_DAY = 'All day'
    FREE = 'Free'
    BOOKED = 'Not free'


class StEvent:
    def __init__(self, event_type: StEventType, t_start: date | datetime, t_end: date | datetime, summary: str,):
        self.event_type = event_type
        self.t_start = t_start if event_type == StEventType.ALL_DAY else self._ensure_datetime(t_start)
        self.t_end = t_end if event_type == StEventType.ALL_DAY else self._ensure_datetime(t_end)
        self.summary = str(summary)

        self.index = self._ensure_datetime(self.t_start)
        

    def __str__(self):
        t_start_str = self.t_start if self.event_type == StEventType.ALL_DAY else self.t_start.strftime('%Y-%m-%d %H:%M')
        t_end_str = self.t_end if self.event_type == StEventType.ALL_DAY else self.t_end.strftime('%Y-%m-%d %H:%M')

        return f"Event Type: {self.event_type}, Start: {t_start_str}, End: {t_end_str}, Summary: {self.summary}"


    def jsonify(self):
        if self.event_type == 'All day':
            return {
                'type': self.event_type,
                't_start': {
                    'year': self.t_start.year,
                    'month': self.t_start.month,
                    'date': self.t_start.day,
                },
                't_end': {
                    'year': self.t_end.year,
                    'month': self.t_end.month,
                    'date': self.t_end.day,
                },
                'summary': self.summary,
            }
        else:
            return {
                'type': self.event_type,
                't_start': self.t_start.timestamp(),
                't_end': self.t_end.timestamp(),
                'summary': self.summary,
            }
    
        
    @staticmethod
    def _ensure_datetime(dt):
        """Make sure the dt is in type of datetime"""        
        if isinstance(dt, datetime):
            dt = dt.replace(tzinfo=None)
        elif isinstance(dt, date):
            dt = datetime(dt.year, dt.month, dt.day)
            
        return dt

In [8]:
def get_events_from_calendar(calendar):
    """"""
    events = []

    for event in calendar.walk('VEVENT'):
        dtstart = event.get('dtstart').dt
        dtend = event.get('dtend').dt

        if isinstance(dtstart, date) and isinstance(dtend, date) and not (isinstance(dtstart, datetime)) and not (isinstance(dtend, datetime)):
            # 全天事件
            # print('')
            # print(dtstart, dtend, event.get('summary'))
            events.append(
                StEvent(StEventType.ALL_DAY, dtstart, dtend, event.get('summary'))
            )

        elif isinstance(dtstart, datetime) and isinstance(dtend, datetime):
            # 常规事件
            # print('')
            # print(dtstart.astimezone(utc_8), dtend.astimezone(utc_8), event.get('summary'))
            events.append(
                StEvent(StEventType.REGULAR, dtstart, dtend, event.get('summary'))
            )

    events = sorted(events, key=lambda event: event.index)
    
    return events


def find_events_in_range(t_start: datetime, t_end: datetime):
    pass

In [9]:
class StCalendar:
    def __init__(self, events=[]):
        self.events = events
        self.free_events = []

    def __str__(self):
        if not self.events:
            return "No events in the calendar."
        events_str = '\n'.join(str(event) for event in self.events)
        return f"Calendar Events:\n{events_str}"

    def find_events_by_type(self, type: StEventType):
        return StCalendar(events=[event for event in self.events if event.event_type == type])
        

    def jsonify(self):
        json_data = []
        for e in self.events:
            json_data.append(e.jsonify())
        return json_data
    
stCalendar = StCalendar(events=get_events_from_calendar(calendar))
stCalendar.jsonify()

[{'type': 'Regular',
  't_start': 1616623200.0,
  't_end': 1616625000.0,
  'summary': 'Writing Center Consultation Session for Tutorial'},
 {'type': 'Regular',
  't_start': 1642160400.0,
  't_end': 1642161600.0,
  'summary': '起床'},
 {'type': 'Regular',
  't_start': 1642162200.0,
  't_end': 1642165200.0,
  'summary': '坐车去东站'},
 {'type': 'Regular',
  't_start': 1642165800.0,
  't_end': 1642169400.0,
  'summary': '安检'},
 {'type': 'Regular',
  't_start': 1642169460.0,
  't_end': 1642209600.0,
  'summary': '坐车'},
 {'type': 'All day',
  't_start': {'year': 2022, 'month': 2, 'date': 21},
  't_end': {'year': 2022, 'month': 2, 'date': 22},
  'summary': '第一周'},
 {'type': 'Regular',
  't_start': 1645373700.0,
  't_end': 1645383600.0,
  'summary': '材料结构与性能（全英文）'},
 {'type': 'Regular',
  't_start': 1645393800.0,
  't_end': 1645403100.0,
  'summary': '统计测试与回归（全英文）'},
 {'type': 'Regular',
  't_start': 1645413600.0,
  't_end': 1645423800.0,
  'summary': '毛泽东思想和中国特色社会主义理论体系概论'},
 {'type': 'Regular',
  